# SQL DATA ANALYSIS TASK

# Context
Fenix is a company that is in renewable energy. To this end, it sells solar kits to populations who are in need. Customers have the option of paying for the kit in cash or on credit. If the customer chooses to buy on credit, he can choose between one year (360 days) or two years (720). He is then asked to pay an amount per day (daily_rate), this amount depending on the duration of the loan and the type of kit he has chosen and corresponds to a day of light. So every time he pays that amount, he has a day of light. After 24 hours the system hangs and it has to make another payment to take advantage of the light. He can decide to pay for more than one day of light. The day number is then the whole part of the amount / daily rate ratio. After this number of days, the system still locks. When a customer spends more than 30 days without light, he is considered inactive. Paid vs Expected (PvE) means the ratio between the total amount paid by the client and the amount he should have paid up to date. It allows to appreciate the recovery.

Let's consider the following tables

Loan_current_daitails

loan_id	|customer_id|daily_rate|days_elapsed|Days_til_lock_out|Total_paid
--------|-----------|----------|------------|-----------------|----------
loan_id_1|customer_id_1|daily_rate_1|days_elapsed_1|Days_til_lock_out_1|Total_paid_1
loan_id_2|customer_id_2|daily_rate_2|days_elapsed_2|Days_til_lock_out_2|Total_paid_2
.........|.............|............|..............|...................|............
loan_id_n|customer_id_n	|daily_rate_n|days_elapsed_n|Days_til_lock_out_n|Total_paid_n


Person_democraphic

customer_id	|country|	region|	profession
------------|-------|---------|-----------
customer_id_1|	Country_1	|Region_1	|profession_1
.............|	............|..........	|............
customer_id_j|	Country_j|	Region_j	|profession_j


Sales_details

loan_id	|Product_type	|Date_fullfiled	|loan_duration
--------|----------------|--------------|-------------
loan_id_1	|Product_type_1	|Date_fullfiled_1	|loan_duration_1
...|	...	|...	|...
loan_id_n|	Product_type_n|	Date_fullfiled_n|	loan_duration_n

### Tables description

* **Loan_id:** unique loan identifier

* **Customer_id:** unique customer identifier

* **Days_elapsed:** the number of days the client spends with us

* **Daily_rate:** expected amount from customer per day

* **Days_til_lock_out:** If positive, then corresponds to the number of days of light remaining before the system hangs. If negative, corresponds to the number of days since the system was blocked (the client has been without light since this number of days)

* **Total_paid:** total amount to be paid by the customer

* **Country:** Customer's country

* **Region:** customer department

* **Profession:** client's profession

* **Product_type:** the type of kit purchased by the customer

* **Loan_duration:** the duration of the customer's loan (360 or 720 days)

* **Date_fullfiled:** the date of the sale

### The datasets
The datasets exist in csv formats and can be downloaded and saved on disk

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [10]:
!pip install ipython-sql

In [1]:
%load_ext sql
%sql sqlite:///testdb.sqlite

'Connected: @testdb.sqlite'

In [2]:
%%sql
DROP TABLE IF EXISTS Loan_current_daitails;
DROP TABLE IF EXISTS Person_democraphic;
DROP TABLE IF EXISTS Sales_details;

 * sqlite:///testdb.sqlite
Done.
Done.
Done.


[]

In [ ]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("testdb.sqlite")
loans = pd.read_csv('/content/Loan_current_daitails.csv')
persons = pd.read_csv('/content/Person_democraphic.csv')
sales = pd.read_csv('/content/Sales_details.csv')

loans.to_sql("loans", conn, index=False, if_exists="replace")
persons.to_sql("persons", conn, index=False, if_exists="replace")
sales.to_sql("sales", conn, index=False, if_exists="replace")

In [11]:

%%sql

select * from persons limit 5


 * sqlite:///testdb.sqlite
Done.


customer_id,country,region,Profession,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
1351176,Benin,Oueme,Tehnician,None,None,None,None
1344957,Benin,Alibori,Doctor,None,None,None,None
1794153,Benin,Oueme,Teacher,None,None,None,None
1291070,Benin,Atlantique,Tehnician,None,None,None,None
1827368,Benin,Oueme,Cmmercial,None,None,None,None


In [12]:
%%sql

select * from sales limit 5

 * sqlite:///testdb.sqlite
Done.


loan_id,product_type,date_fulfilled_utc,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
589556,Fenix 3,6/5/19 10:45,None,None,None,None
586196,Fenix 3,6/2/19 11:11,None,None,None,None
797091,Fenix 2,11/23/19 17:36,None,None,None,None
552403,Fenix Radio +3,5/12/19 10:20,None,None,None,None
812210,Fenix Power 4 + Woofer BJ,12/4/19 10:53,None,None,None,None


In [13]:
%%sql

select * from loans limit 5

 * sqlite:///testdb.sqlite
Done.


loan_id,customer_id,daily_rate,days_elapsed,days_til_lockout,total_paid
589556,1351176,270,265,-13,52355
586196,1344957,160,268,-46,24160
797091,1794153,130,94,6,11000
552403,1291070,175,225,-120,13825
812210,1827368,280,83,5,23000


### Write a SQL code to determine the total number of loans on the one hand and the total number of customers on the other

In [14]:
%%sql
--# each loan has a unique id (loan_id), a count of the ids will indicate total number of loans, 
--# same goes for number of customers
select count(loan_id) number_of_loans, count(customer_id) number_of_customers
from loans

 * sqlite:///testdb.sqlite
Done.


number_of_loans,number_of_customers
5000,5000


### Write a SQL code to determine the total number of loans by type of kit

In [15]:
%%sql

select s.product_type type_of_kit, count(l.loan_id) number_of_loans
from loans as l
join sales as s on l.loan_id = s.loan_id 
group by s.product_type
order by count(l.loan_id) desc

 * sqlite:///testdb.sqlite
Done.


type_of_kit,number_of_loans
Fenix Radio +3,1791
Fenix 3,974
Fenix 2,841
Fenix Power TV 24in FTA BJ,634
Fenix Radio +5,422
"Fenix Power TV 24in FTA Upgrade, BJ",177
Fenix Power 4 + Woofer BJ,35
Fenix Woofer +3,32
"Fenix Power TV 19in FTA Upgrade, BJ",28
Fenix Power 2+ Tondeuse,18


### Write SQL code to determine the number of inactive loans

In [16]:
%%sql
--# inactive loans are indicated by 30 days or more in the dark

select count(loan_id) no_of_inactive_loans
from loans
where days_til_lockout < -30

 * sqlite:///testdb.sqlite
Done.


no_of_inactive_loans
626


### Write a SQL code to determine the regions where the total amount of payments exceeds 1,000,000

In [17]:
%%sql

select p.region, sum(l.total_paid) total_payments
from persons as p
join loans as l on p.customer_id = l.customer_id 
group by p.region
having sum(l.total_paid) > 1000000
order by sum(l.total_paid) desc

 * sqlite:///testdb.sqlite
Done.


region,total_payments
Atlantique,51293780
Oueme,30027194
Borgou,18947292
Collines,11904427
Mono,11621621
Donga,9936982
Plateau,9691206
Zou,9540211
Alibori,8559985
Atacora,7802685


### Write a SQL code to classify by region and in ascending order, the loans according to the total amount paid

In [18]:
%%sql

select p.region, sum(l.total_paid) sum_of_total_paid 
from persons as p
join loans as l on p.customer_id = l.customer_id 
group by p.region
order by sum(l.total_paid) asc

 * sqlite:///testdb.sqlite
Done.


region,sum_of_total_paid
Littoral,3980519
Couffo,4812070
Atacora,7802685
Alibori,8559985
Zou,9540211
Plateau,9691206
Donga,9936982
Mono,11621621
Collines,11904427
Borgou,18947292



### Write a SQL code to determine the top 10 occupations by region with the highest total amount of payments

In [19]:
%%sql
pragma table_info('persons');

 * sqlite:///testdb.sqlite
Done.


cid,name,type,notnull,dflt_value,pk
0,customer_id,INTEGER,0,None,0
1,country,TEXT,0,None,0
2,region,TEXT,0,None,0
3,Profession,TEXT,0,None,0
4,Unnamed: 4,REAL,0,None,0
5,Unnamed: 5,REAL,0,None,0
6,Unnamed: 6,REAL,0,None,0
7,Unnamed: 7,REAL,0,None,0
